## Multi-user, Multi-recording Video Labeler

multiLabelerBatchLabel is designed to allow multiple users to collaboratively label multiple recordings. Before starting, make sure all recordings are converted to a directory of frame-by-frame tiff images.

In [ ]:
from videolabeler import utils as vl
import numpy as np
import cv2
from tqdm.notebook import tqdm
import pandas as pd
import os

Define inputs.

Inputs for multilabeler, in rough order of importance are:

* __root_dir__ : where all your recordings are. Each recording is assumed to have its own subdirectory, named after the animal_id
* __animal_ids__: to indicate which recordings within the root directory you want labeled
* __batch_size__ (default=500): maximum number of frames you want to view and label. Note that you are not allowed to save your labels until you've finished labeling a batch and closed the window.
* __n_overlap_frames__ (default=50): number of overlapping labels for each batch. For instance, if frames 200-400 are already labeled and the user requests a batch of size 500 and overlap 50, they could be given frames 350-850 or frames 0-250, but never frames 0-200 or 400-900. It's recommended to keep a ratio of 1 overlap per 10 frames, so we can get a robust diagnosis of labeler-to-labeler agreement
* __label_dict__: mapping of keys to labels. Provided here for easy reference. Do not adjust for this labeling session.
* __labels_file__ (root_dir/master_labels.csv): where your labels will be saved. Highly recommend keeping it default, since it must be kept consistent for all labelers.
* __min_gap__ (default=200): Smallest allowable gap between data. Let's say we have a 700 frame video and 300 frame batch_size with no overlap, for simplicity. With min_gap=200, videolabeler can give you frames 0-300, __300-500__ or 500-700. Must be smaller than batch_size. Highly recommend keeping above 200 for efficient labeling.

In [ ]:
animal_ids = ['V3-154', 'DCMStr41', 'V3-163']
root_dir = '/ariel/science/mmiller/data/imu_net/tiff_to_label/'

batch_size = 500
n_overlap_frames = 50
label_dict= {'i':'INTERP','s':'still','r':'rearing','w':'walking', 'q':'left turn', 'e':'right turn',
             'a':'left turn [still]', 'd': 'right turn [still]', 'g':'grooming','m':'eating', 't':'explore', 'l':'leap'}

Run videolabeler. Navigation keys will be displayed for your reference in the console.

Note: exit by pressing the key x, instead of using the GUI. Otherwise, the cell hangs and you must interrupt or restart the kernel.

__Interpolation__

You can put the same label on multiple frames quickly using interpolate mode. To do so, activate interpolate mode by pressing i. When interpolate mode is active, every frame after the last label will gain green borders and a gray tentative label. Some frames will have a red label in addition to a gray label, indicating previous labels you will be rewriting. Press i where you want to close this mode. Every frame from the last labeled one to the one you closed interpolate on will now have the same label.

__Example usage__: Animal is walking from frames 10-50. Frame 10 is labeled as 'walking.' Press i anywhere after frame 10 to activate interpolate mode. Press i again on frame 50 to close interpolate mode there. Frames 10-50 are now labeled as 'walking,' and nothing else is touched.

In [ ]:
vl.multiLabelerBatchLabel(root_dir,animal_ids,batch_size=batch_size, n_overlap_frames=n_overlap_frames,min_gap=100)